Biharmonic surfaces are surfaces where the position functions satisfy the biharmonic equation:
 **Δ²x' = 0**


**Subject to certain boundary conditions, denoted as**:
 ***x'b = x'bc***
>
- x': In this equation, x' represents the unknown 3D position of a point on the surface. It's essentially the position vector that defines where each point on the surface is located in three-dimensional space. This vector is unknown because the goal is to find the positions that satisfy the biharmonic equation.

- x'b refers to the position of a point on the surface before any deformation or manipulation.

- x'bc represents the position of the same point on the surface after applying certain boundary conditions or constraints.

## ***Biharmonic Equation (Δ²x' = 0):***
>
- The biharmonic equation is a differential equation that describes how the surface should deform or change.
- It states that the bi-Laplacian of the position vector x' should be zero.
- The bi-Laplacian is an operator that describes how the curvature or bending of the surface should behave.
- **In simple terms, it enforces that the surface remains smooth and doesn't bend or warp excessively.**

in simple words: keeping things super smooth and flat


# =========

### when you see the expression "x'b = x'bc,"

it means that the position of a point on the surface before deformation (x'b) should equal the position of the same point after deformation, subject to the constraints (x'bc).

This is a way to ensure that the surface deformation adheres to specific rules, such as maintaining smoothness (as indicated by the biharmonic equation) and other conditions that may be imposed by the problem or application at hand.

# **Note**
the next part answers the following questions:
- how to perform deformation?
- while ensuring that the rest pose is reproduced
- and that the resulting deformation is smooth 
- and controlled by working with deformation fields and solving for them based on biharmonic equations and handle constraints.

## ***Deformation Fields:***

- Instead of directly manipulating the positions of the vertices on the surface (x), the documentation suggests working with deformation fields (d).
- **Defromation fields**: are essentially displacement vectors. These displacement vectors describe how each point on the surface should move or deform.

#### The deformed positions (x'):
- Obtained by adding the deformation field (d) to the original positions (x). So, x' = x + d.
- i.e. This equation tells you how to calculate the new positions after applying deformation.

#### Smooth Deformation Fields:
- To ensure that the resulting deformed shape (x') is smooth and adheres to certain constraints:
    - recommends using biharmonic deformation fields. These fields satisfy the biharmonic equation: Δ²d = 0, which ensures smoothness.

#### Handle Constraints:
- The deformation fields should respect certain handle constraints. 
- These constraints describe how the handles (control points) move or deform.
- they are written as **db = xbc - xb**
    - **db**: is the deformation field at the boundary (handles), 
    - **xbc**: is the desired position of the handles after deformation, 
    - **xb**: is the original position of the handles.

#### Solving for Deformation Fields: 
- You can use a harmonic solver with k = 2 to find the deformation field that satisfies the biharmonic equation while respecting the handle constraints.
-  After that you can apply it to the original positions (x) to obtain the deformed positions (x').

# Example Code Implementation

In [3]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

In [4]:
v, f = igl.read_triangle_mesh("D:/Medsoft/test/libigl/libgil-examples/decimated-max.obj")
# Swap X and Z axes to adjust the orientation of the mesh as needed.
v[:,[0, 2]] = v[:,[2, 0]] 
u = v.copy()



In [5]:
# read selection matrix s from a file
# This matrix is used to mark certain vertices as handles or constraints for the deformation. 
# It identifies which vertices are influenced by the deformation.
selection_matrix = igl.read_dmat("D:/Medsoft/test/libigl/libgil-examples/decimated-max-selection.dmat") 

# Boundary Conditions
# It identifies the boundary vertices b based on the selection matrix selection_matrix.
# These boundary conditions specify how the selected vertices should move during deformation.
boundary_condition = np.array([[t[0] for t in [(i, selection_matrix[i]) for i in range(0, v.shape[0])] if t[1] >= 0]]).T


In [6]:
## Boundary conditions directly on deformed positions
# sets up u_bc and v_bc to store the boundary conditions for the deformed and original positions.
u_bc = np.zeros((boundary_condition.shape[0], v.shape[1]))
v_bc = np.zeros((boundary_condition.shape[0], v.shape[1]))

for bi in range(boundary_condition.shape[0]):
    # It calculates a deformed mesh using the harmonic deformation technique with a biharmonic operator.
    v_bc[bi] = v[boundary_condition[bi]]

    if selection_matrix[boundary_condition[bi]] == 0: # Don't move handle 0
        u_bc[bi] = v[boundary_condition[bi]]
    elif selection_matrix[boundary_condition[bi]] == 1: # Move handle 1 down
        u_bc[bi] = v[boundary_condition[bi]] + np.array([[0, -50, 0]])
    else: # Move other handles forward
        u_bc[bi] = v[boundary_condition[bi]] + np.array([[-25, 0, 0]])


In [7]:
p = subplot(v, f, selection_matrix, shading={"wireframe": False, "colormap": "tab10"}, s=[1, 4, 0])
for i in range(3):

    # computes an animation frame for boundary conditions
    # (u_bc - v_bc): interpolating between the original boundary conditions (v_bc) and the target boundary conditions (u_bc).
    # This interpolation creates the effect of smooth deformation.
    u_bc_anim = v_bc + i*0.6 * (u_bc - v_bc)
    d_bc = u_bc_anim - v_bc

    # Deformation Calculation
    # v: original mesh
    # f: face information
    # boundary_condition: boundary conditions 
    # d_bc: animated boundary condition
    d = igl.harmonic(v, f, boundary_condition, d_bc, 2)

    # Updating Deformed Mesh
    # is updated by adding the deformation field d to the original vertex positions v
    u = v + d

    # plot
    subplot(u, f, selection_matrix, shading={"wireframe": False, "colormap": "tab10"}, s=[1, 4, i+1], data=p)
p